# Fourth approach

In [1]:
import tensorflow as tf

tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:


import cv2

font = cv2.FONT_HERSHEY_COMPLEX_SMALL

import random

import cv2
import numpy as np
from gym import Env, spaces

font = cv2.FONT_HERSHEY_COMPLEX_SMALL


class Point(object):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        self.x = 0
        self.y = 0
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.name = name

    def set_position(self, x, y):
        self.x = self.clamp(x, self.x_min, self.x_max - self.icon_w)
        self.y = self.clamp(y, self.y_min, self.y_max - self.icon_h)

    def get_position(self):
        return (self.x, self.y)

    def move(self, del_x, del_y):
        self.x += del_x
        self.y += del_y

        self.x = self.clamp(self.x, self.x_min, self.x_max - self.icon_w)
        self.y = self.clamp(self.y, self.y_min, self.y_max - self.icon_h)

    def clamp(self, n, minn, maxn):
        return max(min(maxn, n), minn)


class Chopper(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Chopper, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.cvtColor(cv2.imread("./../graphics/chopper.png"), cv2.COLOR_BGR2GRAY) / 255.0
        # self.icon=gray_img/255.0
        self.icon_w = 10
        self.icon_h = 10
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))
        # self.icon=np.atleast_3d(self.icon)


class Bird(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Bird, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.cvtColor(cv2.imread("./../graphics/bird.png"), cv2.COLOR_BGR2GRAY) / 255.0
        self.icon_w = 8
        self.icon_h = 8
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))
        # self.icon=np.atleast_3d(self.icon)


class Fuel(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Fuel, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.cvtColor(cv2.imread("./../graphics/fuel.png"), cv2.COLOR_BGR2GRAY) / 255.0
        self.icon_w = 8
        self.icon_h = 8
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))
        # self.icon=np.atleast_3d(self.icon)


class ChopperScape(Env):

    def __init__(self):
        super(ChopperScape, self).__init__()

        # 2D observ space
        self.total_reward = 0
        self.observation_shape = (84, 84, 3)

        self.observation_space = spaces.Box(low=np.zeros(self.observation_shape),
                                            high=np.ones(self.observation_shape),
                                            dtype=np.float16)

        # Actions ranging from 0-3
        self.action_space = spaces.Discrete(5)
        self.possible_bird_pos = [82, 75, 65, 55, 45, 35, 25, 15, 5]
        # Create a canvas to render the environment images upon
        self.canvas = np.ones(self.observation_shape) * 1

        # elements from environment
        self.elements = []

        self.max_fuel = 1000

        self.y_min = int(self.observation_shape[0] * 0.1)
        self.x_min = 0

        self.y_max = int(self.observation_shape[0] * 0.9)
        self.x_max = int(self.observation_shape[1])

    def draw_elements_on_canvas(self):
        # Init the canvas
        self.canvas = np.ones(self.observation_shape)
        b, g, r = cv2.split(self.canvas)
        # Draw the elements on the canvas
        for elem in self.elements:
            elem_shape = elem.icon.shape
            x, y = elem.x, elem.y
            b[y: y + elem_shape[1], x:x + elem_shape[0]] = elem.icon
            g[y: y + elem_shape[1], x:x + elem_shape[0]] = elem.icon
            r[y: y + elem_shape[1], x:x + elem_shape[0]] = elem.icon
        self.canvas = cv2.merge([r, g, b])

        text = 'Fuel Left: {} | Rewards: {}'.format(self.fuel_left, self.ep_return)

        # Put the info on canvas
        # self.canvas = cv2.putText(self.canvas, text, (2, 5), font,
        #                           0.8, (0, 0, 0), 1, cv2.LINE_AA)

    def has_collided(self, elem1, elem2):
        x_col = False
        y_col = False

        elem1_x, elem1_y = elem1.get_position()
        elem2_x, elem2_y = elem2.get_position()

        if 2 * abs(elem1_x - elem2_x) <= (elem1.icon_w + elem2.icon_w):
            x_col = True

        if 2 * abs(elem1_y - elem2_y) <= (elem1.icon_h + elem2.icon_h):
            y_col = True

        if x_col and y_col:
            return True

        return False

    def step(self, action):
        # Flag that marks the termination of an episode
        done = False

        # Assert that it is a valid action
        assert self.action_space.contains(action), "Invalid Action"

        # Decrease the fuel counter
        self.fuel_left -= 1

        # Reward for executing a step.
        reward = -0.1

        # apply the action to the chopper
        if action == 0:
            self.chopper.move(0, 2)
        elif action == 1:
            self.chopper.move(0, -2)
        elif action == 2:
            self.chopper.move(2, 0)
        elif action == 3:
            self.chopper.move(-2, 0)
        elif action == 4:
            self.chopper.move(0, 0)

        # Spawn a bird at the right edge with prob 0.01
        if random.random() < 0.019:
            # Spawn a bird
            spawned_bird = Bird("bird".format(self.bird_count), self.x_max, self.x_min, self.y_max, self.y_min)
            self.bird_count += 1

            # Compute the x,y co-ordinates of the position from where the bird has to be spawned
            # Horizontally, the position is on the right edge and vertically, the height is randomly
            # sampled from the set of permissible values
            bird_x = self.x_max
            bird_y = random.choice(self.possible_bird_pos)
            # bird_y = random.randrange(self.y_min, self.y_max)
            spawned_bird.set_position(bird_x, bird_y)

            # Append the spawned bird to the elements currently present in Env.
            self.elements.append(spawned_bird)

            # Spawn a fuel at the bottom edge with prob 0.01
        if random.random() < 0.015:
            # Spawn a fuel tank
            spawned_fuel = Fuel("fuel".format(self.bird_count), self.x_max, self.x_min, self.y_max, self.y_min)
            self.fuel_count += 1

            # Compute the x,y co-ordinates of the position from where the fuel tank has to be spawned
            # Horizontally, the position is randomly chosen from the list of permissible values and
            # vertically, the position is on the bottom edge
            fuel_x = random.randrange(self.x_min + 20, self.x_max - 10)
            fuel_y = self.y_max
            spawned_fuel.set_position(fuel_x, fuel_y)

            # Append the spawned fuel tank to the elements currently present in the Env.
            self.elements.append(spawned_fuel)

            # For elements in the Ev
        for elem in self.elements:
            if isinstance(elem, Bird):
                # If the bird has reached the left edge, remove it from the Env
                if elem.get_position()[0] <= self.x_min:
                    self.bird_count -= 1
                    self.elements.remove(elem)
                else:
                    # Move the bird left by 5 pts.
                    elem.move(-2, 0)

                # If the bird has collided.
                if self.has_collided(self.chopper, elem):
                    # Conclude the episode and remove the chopper from the Env.
                    done = True
                    reward = -100
                    if self.elements.__contains__(self.chopper):
                        self.elements.remove(self.chopper)

            if isinstance(elem, Fuel):
                # If the fuel tank has reached the top, remove it from the Env
                if elem.get_position()[1] <= self.y_min:
                    self.elements.remove(elem)
                else:
                    # Move the Tank up by 5 pts.
                    elem.move(0, -2)

                # If the fuel tank has collided with the chopper.
                if self.has_collided(self.chopper, elem):
                    # Remove the fuel tank from the env.
                    reward = 1
                    if self.elements.__contains__(elem):
                        self.elements.remove(elem)

                    # Fill the fuel tank of the chopper to full.
                    self.fuel_left = self.max_fuel

        # Increment the episodic return
        # self.ep_return += reward

        # Draw elements on the canvas
        self.draw_elements_on_canvas()

        # If out of fuel, end the episode.
        if self.fuel_left == 0:
            done = True

        # actual_state = {}
        # for e in self.elements:
        #     actual_state[abs(e.x - self.chopper.x) + abs(e.y - self.chopper.y)] = (e.x, e.y)

        # print(np.argmin(actual_state))
        self.ep_return += reward
        return self.canvas, reward, done, []

    def reset(self):
        # Reset the fuel consumed
        self.fuel_left = self.max_fuel

        # Reset the reward
        self.ep_return = 0

        # Reset the total_rewards
        self.total_reward = 0
        # Number of birds
        self.bird_count = 0
        self.fuel_count = 0

        # Determine a place to intialise the chopper in
        x = random.randrange(int(self.observation_shape[0] * 0.05), int(self.observation_shape[0] * 0.10))
        y = 50

        # Intialise the chopper
        self.chopper = Chopper("chopper", self.x_max, self.x_min, self.y_max, self.y_min)
        self.chopper.set_position(x, y)

        # Intialise the elements
        self.elements = [self.chopper]

        # Reset the Canvas
        self.canvas = np.ones(self.observation_shape) * 1

        # Draw elements on the canvas
        self.draw_elements_on_canvas()

        # return the observation
        # actual_state = []
        # for e in self.elements:
        #     actual_state.append((e.name, e.x, e.y))

        return self.canvas

    def render(self, mode="human"):
        assert mode in ["human", "rgb_array"], "Invalid mode, must be either \"human\" or \"rgb_array\""
        if mode == "human":
            cv2.imshow("Game", self.canvas)
            cv2.waitKey(10)

        elif mode == "rgb_array":
            return self.canvas

    def close(self):
        cv2.destroyAllWindows()

    def get_action_meanings(self):
        return {0: "Right", 1: "Left", 2: "Down", 3: "Up", 4: "Do Nothing"}






In [3]:
from tqdm import tqdm
import time
import gym
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy, BoltzmannQPolicy
from rl.agents.dqn import DQNAgent

from keras.layers import Dense, Conv2D, MaxPooling2D, Conv1D
from keras.layers import Flatten
from keras.layers import Input
from keras.losses import MeanSquaredError
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import adam_v2
from av7.deep_q_learning import DQN, DDQN, DuelingDQN
import pandas as pd

In [4]:
def build_model(state_space_shape, num_actions, learning_rate):
    model = Sequential()

    model.add(Conv2D(16, (8, 8), (4, 4), input_shape=state_space_shape, activation='relu'))
    model.add(Conv2D(32, (4, 4), (2, 2), activation='relu'))

    model.add(Flatten())

    model.add(Dense(256, activation='relu'))

    model.add(Dense(128, activation='relu'))

    model.add(Dense(num_actions, activation='linear'))

    model.compile(adam_v2.Adam(learning_rate=learning_rate), loss=MeanSquaredError())

    print(model.summary())
    return model

In [4]:
env = ChopperScape()

C:\Users\Mite\miniconda3\envs\tf_gpu\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float16
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [5]:
num_actions = env.action_space.n
learning_rate = 1e-2
state_space_shape = env.observation_space.shape
print(num_actions)
print(state_space_shape)

5
(84, 84, 3)


In [8]:
max_steps = 1000
num_episodes = 10000
batch_size = 16
memory_size = 4096
epsilon = 0.7
discount_factor = 0.99


In [7]:
target_model = build_model(state_space_shape, num_actions, learning_rate)
model = build_model(state_space_shape, num_actions, learning_rate)


NameError: name 'build_model' is not defined

In [13]:
agent = DQN(state_space_shape, num_actions, model, target_model, learning_rate, discount_factor, batch_size,
            memory_size)

In [14]:
episode_summary = []
for episode in tqdm(range(1, num_episodes + 1)):
    state = env.reset()
    done = False
    steps = 0
    total_reward = 0
    while not done and steps < max_steps:
        # t = time.time()
        # print(steps)
        action = agent.get_action(state, epsilon)
        new_state, reward, done, _ = env.step(action)
        # env.render()
        agent.update_memory(state, action, reward, new_state, done)
        state = new_state
        steps += 1
        total_reward += reward
    agent.train()
    agent.update_target_model()
    # epsilon -= decay
    # print(epsilon)
    if steps == max_steps:
        episode_summary.append([episode, total_reward, steps, env.fuel_left, "MAX_STEPS"])
    elif total_reward >= 7:
        episode_summary.append([episode, total_reward, steps, env.fuel_left, "WON"])
    elif env.fuel_left == 0:
        episode_summary.append([episode, total_reward, steps, env.fuel_left, "NO_FUEL"])
    else:
        episode_summary.append([episode, total_reward, steps, env.fuel_left, "DIED"])

    # print(env.ep_return)
    if episode % 2000 == 0:
        agent.save("saved", episode)

df = pd.DataFrame(episode_summary, columns=['Episode', 'Score', 'Steps', 'Remaining_F,uel', 'Cause'])
df.to_csv("DQN_TRAINING_APP4", index=False)

  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\Mite\miniconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
100%|██████████| 10000/10000 [3:12:39<00:00,  1.16s/it]      


In [9]:
agent = DDQN(state_space_shape, num_actions, model, target_model, learning_rate, discount_factor, batch_size,
             memory_size)

In [10]:
episode_summary = []
for episode in tqdm(range(1, num_episodes + 1)):
    state = env.reset()
    done = False
    steps = 0
    total_reward = 0
    while not done and steps < max_steps:
        # t = time.time()
        # print(steps)
        action = agent.get_action(state, epsilon)
        new_state, reward, done, _ = env.step(action)
        # env.render()
        agent.update_memory(state, action, reward, new_state, done)
        state = new_state
        steps += 1
        total_reward += reward
    agent.train()
    agent.update_target_model()
    # epsilon -= decay
    # print(epsilon)
    if steps == max_steps:
        episode_summary.append([episode, total_reward, steps, env.fuel_left, "MAX_STEPS"])
    elif total_reward >= 7:
        episode_summary.append([episode, total_reward, steps, env.fuel_left, "WON"])
    elif env.fuel_left == 0:
        episode_summary.append([episode, total_reward, steps, env.fuel_left, "NO_FUEL"])
    else:
        episode_summary.append([episode, total_reward, steps, env.fuel_left, "DIED"])

    # print(env.ep_return)
    if episode % 2000 == 0:
        agent.save("saved", episode)

df = pd.DataFrame(episode_summary, columns=['Episode', 'Score', 'Steps', 'Remaining_Fuel', 'Cause'])
df.to_csv("DDQN_TRAINING_APP4", index=False)

  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\Mite\miniconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
100%|██████████| 10000/10000 [1:49:05<00:00,  1.53it/s] 


In [9]:
layers = [Conv2D(16, (8, 8), (4, 4), input_shape=state_space_shape, activation='relu'),
          Conv2D(32, (4, 4), (2, 2), activation='relu'),
          Flatten(),
          Dense(256, activation='relu'),
          Dense(128, activation='relu'),
          Dense(num_actions, activation='relu')
          ]

In [10]:
agent = DuelingDQN(state_space_shape, num_actions, learning_rate, discount_factor, batch_size, memory_size)

agent.build_model(layers)

C:\Users\Mite\miniconda3\envs\tf_gpu\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [11]:
episode_summary = []
for episode in tqdm(range(1, num_episodes + 1)):
    state = env.reset()
    done = False
    steps = 0
    total_reward = 0
    if episode == 1000:
        epsilon = 0.5
    elif episode == 1500:
        epsilon = 0.2

    while not done and steps < max_steps:
        # t = time.time()
        # print(steps)
        action = agent.get_action(state, epsilon)
        new_state, reward, done, _ = env.step(action)
        # env.render()
        agent.update_memory(state, action, reward, new_state, done)
        state = new_state
        steps += 1
        total_reward += reward
    agent.train()
    agent.update_target_model()
    # epsilon -= decay
    # print(epsilon)
    if steps == max_steps:
        episode_summary.append([episode, total_reward, steps, env.fuel_left, "MAX_STEPS"])
    elif total_reward >= 7:
        episode_summary.append([episode, total_reward, steps, env.fuel_left, "WON"])
    elif env.fuel_left == 0:
        episode_summary.append([episode, total_reward, steps, env.fuel_left, "NO_FUEL"])
    else:
        episode_summary.append([episode, total_reward, steps, env.fuel_left, "DIED"])

    # print(env.ep_return)
    if episode % 2000 == 0:
        agent.save("saved", episode)

df = pd.DataFrame(episode_summary, columns=['Episode', 'Score', 'Steps', 'Remaining_Fuel', 'Cause'])
df.to_csv("DUELING_TRAINING_APP4", index=False)

  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\Mite\miniconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
100%|██████████| 10000/10000 [1:35:32<00:00,  1.74it/s] 


In [73]:
epsilon = 0.0
testing_results = []
testing_episodes = 5

In [20]:
import pandas as pd

In [74]:

env = ChopperScape()

# env.render()
env.reset()
# env.render()
for i in range(testing_episodes):
    done = False
    state = env.reset()
    # env.render()
    steps = 0
    total_reward = 0
    while not done:
        action = agent.get_action(state, epsilon)
        # action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        # print(action)
        # Render the game
        # print(obs)
        env.render()
        if done == True:
            if steps == max_steps:
                testing_results.append([i, env.ep_return, steps, env.fuel_left, "MAX_STEPS"])
            elif env.ep_return + steps >= 1500:
                testing_results.append([i, env.ep_return, steps, env.fuel_left, "WON"])
            elif env.fuel_left == 0:
                testing_results.append([i, env.ep_return, steps, env.fuel_left, "NO_FUEL"])
            else:
                testing_results.append([i, env.ep_return, steps, env.fuel_left, "DIED"])
            # testing_results.append([i,total_reward,env.fuel_left])
            # print(total_reward)
            break
# print(df)
env.close()

rez = pd.DataFrame(testing_results, columns=["Episode", "Score", "Steps", "Fuel_Left", "Cause"])

C:\Users\Mite\miniconda3\envs\tf_gpu\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float16
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [2]:
import pandas as pd

rez1=pd.read_csv("DQN_TRAINING_APP4")
rez2=pd.read_csv("DUELING_TRAINING_APP4")
rez3=pd.read_csv("DDQN_TRAINING_APP4")

print(rez1['Score'].mean())
print(rez3['Score'].mean())
print(rez2['Score'].mean())

-121.09588000000005
-121.81373000000005
-119.47970000000005
